In [31]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [33]:
%run -i ../../zkstats/core.py

In [34]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [35]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))
# remember that our create_dummy creates random numbers in range 1 - 30, so need to make sure this example the filter doesnt 
# get filter out all data
create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

# # where(element > 20)
gt20_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 20].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt20_dummy_data_tensor)

gt20_data_tensor = data_tensor[data_tensor > 20].reshape(1,-1,1)
theory_output = torch.mean(gt20_data_tensor)


In [36]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        filter = (X>20)
        num_fil_X = torch.sum(filter.double())
        fil_X = torch.where(filter, X, 0.0)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*self.w)<=torch.abs(0.01*num_fil_X*self.w), self.w)
    

verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

In [37]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 20].reshape(1,-1,1)
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        
    def forward(self,X):
        # where part
        filter = (X>20)
        num_fil_X = torch.sum(filter.double())
        fil_X = torch.where(filter, X, 0.0)

        return (torch.abs(torch.sum(fil_X)-num_fil_X*self.w)<=torch.abs(0.01*num_fil_X*self.w), self.w)
    

prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

Theory_output:  tensor(50.2196, dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-154296,1206],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":3318,"total_const_size":306,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [38]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 26.11175513267517 seconds
Theory output:  tensor(50.2196, dtype=torch.float64)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.25
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2359119529189047257, 1186270854317446351, 12298916256516019502, 1116401204062057155]]], 'proof': '2fdcccf3147d0fd78981aacedc04a84fd5f85175fe3f82d29da419f241a79e622777e8afa6fb96db586ace343bfd30b3c714ef283afe03649b62428dcd461a9c0a51ac5443fbf47f630dfbf0097b0846d3ea8272662d047d14363e1becf379782e1fe34bbb01dbdda6fe850372f2f8760a8c8a9c0ef659f6ac9885a5b6e6a28005307dcdc7153f81dd80287f0cad008d5e5ccf9798f87f959e797a6f822e113d1175179b47d9f81149e2852ff55b940f0c337d357b51042953e8652827a626b71151076631997d34c44a03948cedea275fe232d1302f92ff54203999c9e2ca4624e5a9f7b8985e85f4a43d504d556372dbcbb4495661ab1f1cc2f4dd49434c0f251de3691943b7d0afbd76ff1f9f9ce1db9d93a254ab559788a1027bdf7f71d809062d6cf12a15c722006b47c0e9c6f7d4f264336e39a072468ca5166710891d098c93611eda31ca03739aa5c90bd1822aea2d1e251627341cb99473eb0439e

In [39]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2359119529189047257, 1186270854317446351, 12298916256516019502, 1116401204062057155]]]
proof boolean:  1.0
proof result 1 : 50.25
verified
